In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import os
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
from sklearn.preprocessing import MinMaxScaler
import torch.optim as optim
import pathlib
import sys


from sklearn.model_selection import train_test_split



cwd = pathlib.Path().resolve()
src = cwd.parent
root = src.parent
sys.path.append(str(src))

from models.rnn import *
from utils.utils import count_parameters, create_sequence
from utils.train import train_and_validate
from utils.watertopo import WaterTopo
#from training.train import train_and_validate, evaluate_model

In [2]:
#initialize GPU -  In case of windows use cuda instead of mps
device = torch.device('mps')


grid_size = 64

In [3]:
train = True

In [4]:
sim_amount = 80
use_augmented_data = False
T = 4
H = 1
training_size = 0.7
batch_size = 20
num_epochs = 200
num_layers = 1
lr = 0.0005
criterion = nn.MSELoss()
optimizer = optim.AdamW
model_name = "rnn_simple_test"

# defining the save path
save_path = "../results/trained_models/rnn/" + model_name

In [8]:
model = nextgen_rnn(num_layers = num_layers, input_size = (T,64,64) , hidden_size=64, 
                 output_size = (H, 64, 64), sequence_length = T)

TypeError: empty(): argument 'size' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got tuple"

In [ ]:
if train:
    sims = WaterTopo.load_simulations(str(root)+"/data/normalized_data/tra_val", sim_amount=sim_amount, number_grids=64, use_augmented_data=use_augmented_data)

    X, Y = create_sequence(sims, T, H)
    #X, Y = X.squeeze(1), Y.squeeze(1)
    print(X.shape)
    print(Y.shape)

    # We keep track of indexes of train and validation.
    X_tra, X_tst, Y_tra, Y_tst, ix_tra, ix_tst = train_test_split(
        X, Y, np.arange(X.shape[0]), test_size=1-training_size, shuffle=True, random_state=42)

    # Split the existing test dataset into validation and test sets (50/50 split)
    X_val, X_tst, Y_val, Y_tst, ix_val, ix_tst = train_test_split(
        X_tst, Y_tst, ix_tst, test_size=0.5, shuffle=True, random_state=42)

    #create datasets and data loaders
    train_dataset = TensorDataset(torch.tensor(X_tra, dtype=torch.float32), torch.tensor(Y_tra, dtype=torch.float32))
    val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(Y_val, dtype=torch.float32))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # defining the optimizer
    optimizer = optimizer(model.parameters(), lr=lr)

    # training
    train_losses, val_losses, best_val_loss, time = train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, save_path)

# Load the best model
model.load_state_dict(torch.load(save_path, map_location="cpu"))

In [ ]:
X = np.zeros((96, 4096))
Y = np.zeros((96, 4096))

topography = sim.topography

for t_i in range(len(X)):
    wd, vx, vy = sim.return_timestep(t_i)
    X[t_i] = wd.reshape(-1)

    wd_vx_vy = sim.return_timestep(t_i+1)
    Y[t_i] = wd.reshape(-1)

training_size = 0.8
id_training = int(training_size * len(X))

X_tra = X[:id_training, :]
Y_tra = Y[:id_training, :]

X_val = X[id_training:, :]
Y_val = Y[id_training:, :]
# print(np.count_nonzero(X_tra[95]))
print(X_tra.shape)
print(Y_tra.shape)
print(X_val.shape)
print(Y_val.shape)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:

train_dataset = TensorDataset(torch.tensor(X_tra, dtype=torch.float32), torch.tensor(Y_tra, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(Y_val, dtype=torch.float32))

In [ ]:
# # Instantiate the model with hyperparameters
# model = RNN_floods(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
# # We'll also set the model to the device that we defined earlier (default is CPU)
# model.to(device)

# # Define hyperparameters
# n_epochs = 100
# lr=0.01

# # Define Loss, Optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
batch_size = 2**2      # You can modify this based on your requirements

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
num_epochs = 200
lr = 0.0005

In [ ]:
import torch.optim as optim

In [ ]:
babies_first_model = (128, 4096).to(device)

In [ ]:
# Loss function
criterion = nn.MSELoss()  # Or another appropriate loss function

optimizer = optim.AdamW(babies_first_model.parameters(), lr=lr)
save_path_MLP = '../trained_models/RNN/RNN_v2.pth'

In [ ]:
import time

In [ ]:
train_losses_MLP, val_losses_MLP, best_val_loss_MLP, time_MLP = train_and_validate(babies_first_model, train_loader, val_loader, criterion, optimizer, num_epochs, device, save_path_MLP)